## Data Collection and Cleaning
### Collection
First, information about every IHSA member school was inserted into a dictionary called school_facts, which compiled all relevant information about the school (coordinates, size, etc.) into a single dictionary entry by the school's name (the IHSA formats each school so no two schools share the same name). Every IHSA member school has an ID with a fact page that can be accessed with the following formatted link, where "[ID]" is replaced with the four-digit ID: "https://www.ihsa.org/data/school/schools/[ID].htm".

In [2]:
%%time

#should take approximately 7 minutes to run, at most 8:22.

import pandas as pd
import numpy as np
import matplotlib
import requests
from bs4 import BeautifulSoup
import io
from csv import writer
import re

#fetches IHSA school data and inputs into dict named school_facts

school_facts = {}

for id_int in range(1, 3000): #all possible IHSA school IDs
    if id_int in [431, 733, 1650, 1651, 2849, 2970, 2972]: continue #these were non-IHSA member schools which cannot compete in IHSA activities
    ID = "{:04}".format(id_int)
    response = requests.get('https://www.ihsa.org/data/school/schools/' + ID + '.htm')
    wholePage = BeautifulSoup(response.text, 'html.parser')
    
    if wholePage.find('i') is not None:
        #this section gets the coordinates of each school by extracting them from the Google Maps link
        bigLink = wholePage.find('i').find('a')['href']
        coords = bigLink[bigLink.find('&q=') + 3:bigLink.find('%20(')]
        lat = float(coords[:coords.find(',')])
        long = float(coords[coords.find(',')+1:])
        exact = True
        
        #this checks if school size is can be determined and records it if it can
        check = False
        for guess in range(15-2, 15+3): 
            temp = wholePage.find_all('p')[guess].get_text().strip()
            if temp.find('Enrollment') != -1:
                sizeComplex = temp
                check = True
        if check:
            if sizeComplex.find('/') != -1: size = float(sizeComplex[12:sizeComplex.find(' /')])
            else: size = float(sizeComplex[12:])
        else: size = None
        
        #this checks if the conference can be determined and records it if it can
        check = False
        for guess in range(16-2, 16+3):
            temp = wholePage.find_all('p')[guess].get_text().strip()
            if temp.find('Conference') != -1:
                conf = temp[15:]
                check = True
        if not check: conf = None
        
        #this checks if the school type (e.g. co-ed) can be determined and records it if it can
        check = False
        for guess in range(19-2, 19+3):
            temp = wholePage.find_all('p')[guess].get_text().strip()
            if temp.find('School Type') != -1:
                type_var = temp[13:]
                check = True
        if not check: type_var = None
        
        #this checks if the county can be determined and records it if it can
        check = False
        for guess in range(20-2, 20+3):
            temp = wholePage.find_all('p')[guess].get_text().strip()
            if temp.find('County') != -1:
                county = temp[8:]
                check = True
        if not check: county = None
        
        #this checks if the school has board information and records it if it can
        check = False
        for guess in range(22-2, 22+3):
            temp = wholePage.find_all('p')[guess].get_text().strip()
            if temp.find('Board') != -1:
                bigBoy22 = temp
                check = True
        if check:
            board = bigBoy22[:bigBoy22.find(', ')]
            legis = bigBoy22[bigBoy22.find(', ')+2:]
            brd_div = int(board[board.find(': ') + 2:])
            lg_dt = int(legis[legis.find(': ') + 2:])
        else:
            brd_div = lg_dt = None
            
        #all schools found in this fashion are open
        open_q = True
        
        #records the information gathered into a dictionary, to be appended into the larger dictionary school_facts
        this_school = {}
        this_school["ID"] = ID
        this_school["lat"] = lat
        this_school["long"] = long
        this_school["exact"] = exact
        this_school["size"] = size
        this_school["conf"] = conf
        this_school["type_var"] = type_var
        this_school["county"] = county
        this_school["brd_div"] = brd_div
        this_school["lg_dt"] = lg_dt
        this_school["open_q"] = open_q
        
        #appends the school information into school_facts based on the school's name
        school_facts[wholePage.find('h1').get_text()] = this_school

Wall time: 8min 22s


Next, every school which had coordinates that could not be determined by referencing the name of the city it resided in (usually very small cities, or Chicago by design) had to be manually determined. Each school was identified after a first pass of the data collection process (the cells further below this one). To place the most probable location of the school when it existed (as mot schools that could not be determined were schools that no longer existed), I manually cross-referenced the respective school websites, if they still existed, with information compiled from http://www.illinoishsglorydays.com/ and Wikipedia. Although these websites are unreliable and compiled by anonymous individuals, each manually-determined set of coordinates was checked on Google Maps street view and Google Images to see if the buildings look similar to historic photos, for example, and it is highly unlikely that any building is listed as more than a mile from where it truly was. The schools that were manually entered were determined to have coordinates as follows:

|                 School                     | Lat         | Long         |
|--------------------------------------|-------------|--------------|
| Averyville                           | 40.708451   | -89.572593   |
| Benton Harbor, Mich.                 | 42.102444   | -86.457077   |
| Brazil, Ind.                         | 39.526263   | -87.126977   |
| Chicago (Austin Polytechnic)         | 41.88522421 | -87.76278634 |
| Chicago (Calumet)                    | 41.7381     | -87.5729     |
| Chicago (Carver)                     | 41.6567     | -87.5892     |
| Chicago (Collins)                    | 41.86364443 | -87.70081799 |
| Chicago (Crane)                      | 41.8768     | -87.6832     |
| Chicago (Englewood)                  | 41.782002   | -87.63498    |
| Chicago (English)                    | 41.8768     | -87.6832     |
| Chicago (Good Counsel)               | 41.990945   | -87.72774    |
| Chicago (Gordon Tech)                | 41.9479     | -87.6974     |
| Chicago (Hales Franciscan)           | 41.805      | -87.606944   |
| Chicago (Harrison)                   | 41.848639   | -87.697222   |
| Chicago (Harvard)                    | 41.809804   | -87.601895   |
| Chicago (John Hope)                  | 41.793338   | -87.640949   |
| Chicago (Lewis Institute)            | 41.88068    | -87.674441   |
| Chicago (Lourdes)                    | 41.835609   | -87.653011   |
| Chicago (Luther North)               | 41.9575     | -87.770556   |
| Chicago (Luther South)               | 41.735917   | -87.700611   |
| Chicago (Manual)                     | 41.867483   | -87.62412    |
| Chicago (Maria)                      | 41.770833   | -87.6925     |
| Chicago (McKinley)                   | 41.879054   | -87.67846    |
| Chicago (Mendel)                     | 41.694075   | -87.615897   |
| Chicago (Near North)                 | 41.90858938 | -87.6443343  |
| Chicago (New Phillips)               | 41.82448    | -87.61945    |
| Chicago (North Division)             | 41.9194     | -87.6456     |
| Chicago (Northwest Division)         | 41.90608    | -87.685463   |
| Chicago (Parker)                     | 41.9227     | -87.6379     |
| Chicago (Providence-St. Mel)         | 41.879196   | -87.715428   |
| Chicago (Pullman Manual)             | 41.694      | -87.6159     |
| Chicago (Quigley North)              | 41.897778   | -87.625833   |
| Chicago (Quigley Prep)               | 41.897778   | -87.625833   |
| Chicago (Quigley South)              | 41.752528   | -87.684778   |
| Chicago (Robeson)                    | 41.7704     | -87.6363     |
| Chicago (South Division)             | 41.82448    | -87.61945    |
| Chicago (South Shore)                | 41.7561     | -87.5794     |
| Chicago (St. Benedict)               | 41.9525     | -87.683889   |
| Chicago (St. Gregory)                | 41.983056   | -87.671389   |
| Chicago (St. Martin de Porres)       | 42.35198704 | -87.83256582 |
| Chicago (St. Mary of Perpetual Help) | 41.83550658 | -87.65326308 |
| Chicago (St. Michael)                | 41.745852   | -87.547685   |
| Chicago (Tabernacle Christian)       | 41.69465755 | -87.65332186 |
| Chicago (Tuley)                      | 41.90605435 | -87.68549739 |
| Chicago (VOISE Academy)              | 41.88522421 | -87.76278634 |
| Chicago (Waller)                     | 41.9194     | -87.6456     |
| Chicago (Weber)                      | 41.920553   | -87.757894   |
| Chicago (West Division)              | 41.879054   | -87.67846    |
| Chicago (Westinghouse)               | 41.8891     | -87.7084     |
| Dundee                               | 42.107936   | -88.274486   |
| Georgetown                           | 39.972679   | -87.641228   |
| LaHarpe                              | 40.584113   | -90.974642   |
| Milwaukee, Wis. (South Division)     | 43.013405   | -87.930587   |
| Oregon [Coop]                        | 42.142145   | -89.291035   |
| Shawneetown                          | 37.70975878 | -88.16549612 |
| St. Joseph, Mich.                    | 42.089537   | -86.491634   |
| St. Louis, Mo. (Central)             | 38.65943    | -90.215331   |
| Chicago (Academy of Our Lady)        | 41.7202     | -87.656308   |

In [3]:
#populates manual_schools with manually found closed schools (usually Chicago) 
manual_xl = pd.read_excel("Manually Entered School Coordinates.xlsx", index_col = 0).to_dict()

manual_schools = {}
for key in manual_xl.keys():
    this_school = {}
    this_school["ID"] = None
    this_school["lat"] = manual_xl[key]["Lat"]
    this_school["long"] = manual_xl[key]["Long"]
    this_school["exact"] = False
    this_school["size"] = None
    this_school["conf"] = None
    this_school["type_var"] = None
    this_school["county"] = None
    this_school["brd_div"] = None
    this_school["lg_dt"] = None
    this_school["open_q"] = False
    
    manual_schools[key] = this_school

In [4]:
leftover_schools = {}

#pulls info from school_facts or leftover_schools, if not in either it populates leftover_schools
def get_give_school_info(school):
    if school in school_facts:
        ID = school_facts[school]["ID"]
        lat = school_facts[school]["lat"]
        long = school_facts[school]["long"]
        exact = school_facts[school]["exact"]
        size = school_facts[school]["size"]
        conf = school_facts[school]["conf"]
        type_var = school_facts[school]["type_var"]
        county = school_facts[school]["county"]
        brd_div = school_facts[school]["brd_div"]
        lg_dt = school_facts[school]["lg_dt"]
        open_q = school_facts[school]["open_q"]
    elif school in leftover_schools:
        ID = leftover_schools[school]["ID"]
        lat = leftover_schools[school]["lat"]
        long = leftover_schools[school]["long"]
        exact = leftover_schools[school]["exact"]
        size = leftover_schools[school]["size"]
        conf = leftover_schools[school]["conf"]
        type_var = leftover_schools[school]["type_var"]
        county = leftover_schools[school]["county"]
        brd_div = leftover_schools[school]["brd_div"]
        lg_dt = leftover_schools[school]["lg_dt"]
        open_q = leftover_schools[school]["open_q"]
    elif school in manual_schools:
        ID = manual_schools[school]["ID"]
        lat = manual_schools[school]["lat"]
        long = manual_schools[school]["long"]
        exact = manual_schools[school]["exact"]
        size = manual_schools[school]["size"]
        conf = manual_schools[school]["conf"]
        type_var = manual_schools[school]["type_var"]
        county = manual_schools[school]["county"]
        brd_div = manual_schools[school]["brd_div"]
        lg_dt = manual_schools[school]["lg_dt"]
        open_q = manual_schools[school]["open_q"]
    else:
        exact = False
        goUpTo = school.find("(") - 1
        if goUpTo == -2: goUpTo = len(school)
        extension = school[:goUpTo].replace(" ", "_")
        if extension != 'Chicago':
            extension = extension.replace('Mt.','Mount')
            response = requests.get('https://citylatitudelongitude.com/IL/' + extension + '.htm')
            wholePage = BeautifulSoup(response.text, 'html.parser')
            try:
                html = wholePage.find_all('table')[1].find_all('strong')
                lat = float(html[0].get_text()[2:].strip())
                long = float(html[1].get_text()[2:].strip())
                exact = True
            except IndexError:
                lat = long = None
                pass
        else: lat = long = None

        ID = brd_div = lg_dt = None
        size = conf = type_var = county = None
        open_q = False

        this_school = {}
        this_school["ID"] = ID
        this_school["lat"] = lat
        this_school["long"] = long
        this_school["exact"] = exact
        this_school["size"] = size
        this_school["conf"] = conf
        this_school["type_var"] = type_var
        this_school["county"] = county
        this_school["brd_div"] = brd_div
        this_school["lg_dt"] = lg_dt
        this_school["open_q"] = open_q

        leftover_schools[school] = this_school

    return ID, lat, long, exact, size, conf, type_var, county, brd_div, lg_dt, open_q

Finally, the actual tables of data for acttivity results were parsed into csv file with the following code.

In [5]:
%%time

#reads in the data from non-tournament based activities and (1) Drama and (2) Group Interpretation
#prints what activity it is presently reading

nontournaments = ["bwb", "ccb", "gob", "gyb", "swb", "tnb", "trb", "wr", "bad", "bwl", "chc", "ccg", "gog", "gyg", "swg", "tng", "trg", "dac", "bsf", "ct", "ie", "jrn", "mu"] 

with io.open('IHSA_Deposit.csv', 'w', encoding='utf-8') as csv_file:
    csv_writer = writer(csv_file)
    csv_writer.writerow(['ABBR', 'DIV', 'YEAR', 'PLACE', 'TOURNAMENT', 'SCORE', 'SCHOOL', 'ID', 'LAT', 'LONG', 'EXACT', 'SIZE', 'CONF', 'TYPE', 'COUNTY', 'BRD-DIV', 'LG-DT', 'OPEN'])
    
    for activity in nontournaments:
        print(activity)
        response = requests.get('https://www.ihsa.org/data/' + activity + '/records/wbyyr.htm')
        wholePage = BeautifulSoup(response.text, 'html.parser')

        preHeadings = wholePage.find_all('h3')
        headings = []
        for heading in preHeadings: headings.append(heading.get_text().strip())
        tables = wholePage.find_all('table')
        results = []
        for table in tables:
            rows = table.find_all('tr')
            preResults = []
            for row in rows[2:]:
                info = row.find_all('td')

                score_literal = info[2].get_text()
                if info[2].find('small') != None:
                    score_int, score_frac = info[2].contents
                    score_int = int(score_int)
                    numerator = int(score_frac.find('sup').get_text())
                    denominator = int(score_frac.find('sub').get_text())
                    score_literal = score_int + numerator/denominator
                elif score_literal == '\xa0':
                    score_literal = None
                elif score_literal[-1] == "?":
                    score_literal = float(score_literal[:-1])
                elif re.fullmatch("\d+\s+\d+\/\d+", score_literal): #e.g. 112 31/14
                    score_int = int(re.search("\d+(?=\s)", score_literal).group())
                    numerator = int(re.search("(?<=\s)\d+(?=\/)", score_literal).group())
                    denominator = int(re.search("(?<=\/)\d+", score_literal).group())
                    score_literal = score_int + numerator/denominator
                elif re.fullmatch("\d+\D\d+", score_literal): #e.g. 190-2, 190:2
                    score_literal = float(re.search("\d+(?=\D)", score_literal).group())
                elif re.search("\d+\spt", score_literal): #e.g. 8 pts.
                    score_literal = re.search(".+(?=\spt)", score_literal).group()
                    if re.search("\d+\/\d+", score_literal): #e.g. 31/14
                        numerator = int(re.search("\d+(?=\/)", score_literal).group())
                        denominator = int(re.search("(?<=\/)\d+", score_literal).group())
                        score_literal = numerator/denominator

                if score_literal is not None: score_literal = float(score_literal)
                preResults.append([info[0].get_text().strip(), info[1].get_text(), score_literal])
            results.append(preResults)

        for num in range(len(results)):
            yearCombine = results[num]
            garble = headings[num]
            dash = garble.find('-')
            abbr = activity
            year = int(garble[(dash - 4):(dash)]) + 1
            div = garble[(dash + 4):]
            if div == "": div = "N/A"

            for row in yearCombine:
                place = int(row[0])
                school = row[1]
                if re.match("\d", school[0]) is not None: school = school[1:]
                score = row[2]
                ID, lat, long, exact, size, conf, type_var, county, brd_div, lg_dt, open_q = get_give_school_info(school)
                
                temp = [abbr, div, year, place, False, score, school, ID, lat, long, exact, size, conf, type_var, county, brd_div, lg_dt, open_q]
                csv_writer.writerow(temp)
                
with io.open('IHSA_Deposit.csv', 'a', encoding='utf-8') as csv_file:
    csv_writer = writer(csv_file)
    
    for activity in ["dr", "gi"]:
        print(activity)
        response = requests.get('https://www.ihsa.org/data/dgi/records/' + activity + '/wbyyr.htm')
        wholePage = BeautifulSoup(response.text, 'html.parser')

        preHeadings = wholePage.find_all('h3')
        headings = []
        for heading in preHeadings: headings.append(heading.get_text().strip())
        tables = wholePage.find_all('table')
        results = []
        for table in tables:
            rows = table.find_all('tr')
            preResults = []
            for row in rows[2:]:
                info = row.find_all('td')

                score_literal = info[2].get_text()
                if info[2].find('small') != None:
                    score_int, score_frac = info[2].contents
                    score_int = int(score_int)
                    numerator = int(score_frac.find('sup').get_text())
                    denominator = int(score_frac.find('sub').get_text())
                    score_literal = score_int + numerator/denominator
                elif score_literal == '\xa0':
                    score_literal = None
                elif score_literal[-1] == "?":
                    score_literal = float(score_literal[:-1])
                elif re.fullmatch("\d+\s+\d+\/\d+", score_literal): #e.g. 112 31/14
                    score_int = int(re.search("\d+(?=\s)", score_literal).group())
                    numerator = int(re.search("(?<=\s)\d+(?=\/)", score_literal).group())
                    denominator = int(re.search("(?<=\/)\d+", score_literal).group())
                    score_literal = score_int + numerator/denominator
                elif re.fullmatch("\d+\D\d+", score_literal): #e.g. 190-2, 190:2
                    score_literal = float(re.search("\d+(?=\D)", score_literal).group())
                elif re.search("\d+\spt", score_literal): #e.g. 8 pts.
                    score_literal = re.search(".+(?=\spt)", score_literal).group()
                    if re.search("\d+\/\d+", score_literal): #e.g. 31/14
                        numerator = int(re.search("\d+(?=\/)", score_literal).group())
                        denominator = int(re.search("(?<=\/)\d+", score_literal).group())
                        score_literal = numerator/denominator

                if score_literal == "A": score_literal = 2
                elif score_literal == "B": score_literal = 1
                elif score_literal is not None: score_literal = float(score_literal)
                preResults.append([info[0].get_text().strip(), info[1].get_text(), score_literal])
            results.append(preResults)

        for num in range(len(results)):
            yearCombine = results[num]
            garble = headings[num]
            dash = garble.find('-')
            abbr = activity
            year = int(garble[(dash - 4):(dash)]) + 1
            div = garble[(dash + 4):]
            if div == "": div = "N/A"

            for row in yearCombine:
                place = int(row[0])
                school = row[1]
                if re.match("\d", school[0]) is not None: school = school[1:]
                score = row[2]
                ID, lat, long, exact, size, conf, type_var, county, brd_div, lg_dt, open_q = get_give_school_info(school)
                
                temp = [abbr, div, year, place, False, score, school, ID, lat, long, exact, size, conf, type_var, county, brd_div, lg_dt, open_q]
                csv_writer.writerow(temp)

bwb
ccb
gob
gyb
swb
tnb
trb
wr
bad
bwl
chc
ccg
gog
gyg
swg
tng
trg
dac
bsf
ct
ie
jrn
mu
dr
gi
Wall time: 1min 32s


In [6]:
%%time

#reads in the data from tournament-based activities
tournaments = ["ba", "lcb", "sob", "wpb", "wr", "bkg", "lcg", "sog", "sbg", "wpg", "scb"] 

with io.open('IHSA_Deposit.csv', 'a', encoding='utf-8') as csv_file:
    csv_writer = writer(csv_file)
    
    for activity in tournaments:
        print(activity)
        response = requests.get('https://www.ihsa.org/data/' + activity + '/records/xbyyr1.htm')
        wholePage = BeautifulSoup(response.text, 'html.parser')

        preHeadings = wholePage.find_all('h2')
        
        for heading in preHeadings:
            prevline = heading
            thisline = prevline.next_element.next_element.next_element
            score = "ERROR!"
            block_schools = {}
            points = 0
            while thisline.name != "h2":
                if 'b' in thisline['class']:
                    if thisline.get_text().strip() != "Pool 2": points += 1
                    score = thisline.get_text().strip()
                else:
                    if score == "Championship":
                        block_schools[re.search(".+(?=\s+\d+,)",thisline.get_text().strip()).group()] = {'place': "Champion", 'points': points + 1}
                        block_schools[re.search("(?<=,\s).+(?=\s+\d+(?!\/))",thisline.get_text().strip()).group()] = {'place': "Runner-Up", 'points': points}
                    elif score == "Third Place":
                        block_schools[re.search(".+(?=\s+\d+,)",thisline.get_text().strip()).group()] = {'place': "Third", 'points': points + 1}
                        block_schools[re.search("(?<=,\s).+(?=\s+\d+(?!\/))",thisline.get_text().strip()).group()] = {'place': "Fourth", 'points': points}
                    else:
                        block_schools[re.search(".+(?=\s+\d+,)",thisline.get_text().strip()).group()] = {'place': score, 'points': points}
                        block_schools[re.search("(?<=,\s).+(?=\s+\d+(?!\/))",thisline.get_text().strip()).group()] = {'place': score, 'points': points}
                prevline = thisline
                thisline = prevline.next_element.next_element.next_element
                if type(prevline) is not type(thisline): break
            garble = heading.get_text().strip()
            abbr = activity
            year = int(garble[:4]) + 1
            paren = garble.find('(')
            if paren != -1: div = garble[paren+1: garble.find(')')]
            else: div = "N/A"

            for school in block_schools.keys():
                place = block_schools[school]['place']
                score = block_schools[school]['points']
                if re.match("\d", school[0]) is not None: school = school[1:]
                ID, lat, long, exact, size, conf, type_var, county, brd_div, lg_dt, open_q = get_give_school_info(school)
                
                temp = [abbr, div, year, place, True, score, school, ID, lat, long, exact, size, conf, type_var, county, brd_div, lg_dt, open_q]
                csv_writer.writerow(temp)
                
tournaments = ["vbb", "vbg"] 

with io.open('IHSA_Deposit.csv', 'a', encoding='utf-8') as csv_file:
    csv_writer = writer(csv_file)
    
    for activity in tournaments:
        print(activity)
        response = requests.get('https://www.ihsa.org/data/' + activity + '/records/xbyyr1.htm')
        wholePage = BeautifulSoup(response.text, 'html.parser')

        preHeadings = wholePage.find_all('h2')
        
        for heading in preHeadings:
            prevline = heading
            thisline = prevline.next_element.next_element.next_element
            score = "ERROR!"
            block_schools = {}
            points = 0
            while thisline.name != "h2":
                if 'b' in thisline['class']:
                    points += 1
                    score = thisline.get_text().strip()
                else:
                    if score == "Championship":
                        block_schools[re.search(".+(?=\s+d\.\s)",thisline.get_text().strip()).group()] = {'place': "Champion", 'points': points + 1}
                        block_schools[re.search("(?<=d\.\s).+(?=,)(?<=\D)",thisline.get_text().strip()).group()] = {'place': "Runner-Up", 'points': points}
                    elif score == "Third Place":
                        block_schools[re.search(".+(?=\s+d\.\s)",thisline.get_text().strip()).group()] = {'place': "Third", 'points': points + 1}
                        block_schools[re.search("(?<=d\.\s).+(?=,)(?<=\D)",thisline.get_text().strip()).group()] = {'place': "Fourth", 'points': points}
                    else:
                        block_schools[re.search(".+(?=\s+d\.\s)",thisline.get_text().strip()).group()] = {'place': score, 'points': points}
                        block_schools[re.search("(?<=d\.\s).+(?=,)(?<=\D)",thisline.get_text().strip()).group()] = {'place': score, 'points': points}
                prevline = thisline
                thisline = prevline.next_element.next_element.next_element
                if type(prevline) is not type(thisline): break
            garble = heading.get_text().strip()
            abbr = activity
            year = int(garble[:4]) + 1
            paren = garble.find('(')
            if paren != -1: div = garble[paren+1: garble.find(')')]
            else: div = "N/A"

            for school in block_schools.keys():
                place = block_schools[school]['place']
                score = block_schools[school]['points']
                if re.match("\d", school[0]) is not None: school = school[1:]
                ID, lat, long, exact, size, conf, type_var, county, brd_div, lg_dt, open_q = get_give_school_info(school)
                
                temp = [abbr, div, year, place, True, score, school, ID, lat, long, exact, size, conf, type_var, county, brd_div, lg_dt, open_q]
                csv_writer.writerow(temp)

ba
lcb
sob
wpb
wr
bkg
lcg
sog
sbg
wpg
scb
vbb
vbg
Wall time: 12.1 s


In [7]:
%%time

#reads in the specially-formatted data from Boys Basketball and Boys Football
with io.open('IHSA_Deposit.csv', 'a', encoding='utf-8') as csv_file:
    csv_writer = writer(csv_file)
    
    activity = "bkb"
    print(activity)
    for i in range(1, 5):
        response = requests.get('https://www.ihsa.org/data/' + activity + '/records/xbyyr' + str(i) + '.htm')
        wholePage = BeautifulSoup(response.text, 'html.parser')

        preHeadings = wholePage.find_all('h2')

        for heading in preHeadings:
            prevline = heading
            thisline = prevline.next_element.next_element.next_element
            score = "ERROR!"
            block_schools = {}
            points = 0
            while thisline.name != "h2":
                if 'b' in thisline['class']:
                    if thisline.get_text().strip() != "Consolation Round": points += 1
                    score = thisline.get_text().strip()
                else:
                    if score == "Championship":
                        block_schools[re.search(".+(?=\s+\d+,)",thisline.get_text().strip()).group()] = {'place': "Champion", 'points': points + 1}
                        block_schools[re.search("(?<=,\s).+(?=\s+\d+)",thisline.get_text().strip()).group()] = {'place': "Runner-Up", 'points': points}
                    elif score == "Third Place":
                        block_schools[re.search(".+(?=\s+\d+,)",thisline.get_text().strip()).group()] = {'place': "Third", 'points': points + 1}
                        block_schools[re.search("(?<=,\s).+(?=\s+\d+)",thisline.get_text().strip()).group()] = {'place': "Fourth", 'points': points}
                    else:
                        block_schools[re.search(".+(?=\s+\d+,)",thisline.get_text().strip()).group()] = {'place': score, 'points': points}
                        block_schools[re.search("(?<=,\s).+(?=\s+\d+)",thisline.get_text().strip()).group()] = {'place': score, 'points': points}
                prevline = thisline
                thisline = prevline.next_element.next_element.next_element
                if type(prevline) is not type(thisline): break
            garble = heading.get_text().strip()
            abbr = activity
            year = int(garble[:4]) + 1
            paren = garble.find('(')
            if paren != -1: div = garble[paren+1: garble.find(')')]
            else: div = "N/A"

            for school in block_schools.keys():
                place = block_schools[school]['place']
                score = block_schools[school]['points']
                if re.match("\d", school[0]) is not None: school = school[1:]
                ID, lat, long, exact, size, conf, type_var, county, brd_div, lg_dt, open_q = get_give_school_info(school)
                
                temp = [abbr, div, year, place, True, score, school, ID, lat, long, exact, size, conf, type_var, county, brd_div, lg_dt, open_q]
                csv_writer.writerow(temp)
                
    activity = "fb"
    print(activity)
    for i in range(1, 9):
        response = requests.get('https://www.ihsa.org/data/' + activity + '/records/xbyyr' + str(i) + '.htm')
        wholePage = BeautifulSoup(response.text, 'html.parser')

        preHeadings = wholePage.find_all('h2')

        for heading in preHeadings:
            prevline = heading
            thisline = prevline.next_element.next_element.next_element
            score = "ERROR!"
            block_schools = {}
            points = 0
            while thisline.name != "h2":
                if 'b' in thisline['class']:
                    points += 1
                    score = thisline.get_text().strip()
                else:
                    if score == "Championship":
                        block_schools[re.search(".+(?=\s+\d+,)",thisline.get_text().strip()).group().replace("@", '')] = {'place': "Champion", 'points': points + 1}
                        block_schools[re.search("(?<=,\s).+(?=\s+\d+)",thisline.get_text().strip()).group().replace("@", '')] = {'place': "Runner-Up", 'points': points}
                    elif score == "Third Place":
                        block_schools[re.search(".+(?=\s+\d+,)",thisline.get_text().strip()).group().replace("@", '')] = {'place': "Third", 'points': points + 1}
                        block_schools[re.search("(?<=,\s).+(?=\s+\d+)",thisline.get_text().strip()).group().replace("@", '')] = {'place': "Fourth", 'points': points}
                    else:
                        block_schools[re.search(".+(?=\s+\d+,)",thisline.get_text().strip()).group().replace("@", '')] = {'place': score, 'points': points}
                        block_schools[re.search("(?<=,\s).+(?=\s+\d+)",thisline.get_text().strip()).group().replace("@", '')] = {'place': score, 'points': points}
                prevline = thisline
                thisline = prevline.next_element.next_element.next_element
                if thisline == "(forfeited)":
                    thisline = thisline.next_element.next_element
                if type(prevline) is not type(thisline): break
            garble = heading.get_text().strip()
            abbr = activity
            year = int(garble[:4]) + 1
            paren = garble.find('(')
            if paren != -1: div = garble[paren+1: garble.find(')')]
            else: div = "N/A"

            for school in block_schools.keys():
                place = block_schools[school]['place']
                score = block_schools[school]['points']
                if re.match("\d", school[0]) is not None: school = school[1:]
                ID, lat, long, exact, size, conf, type_var, county, brd_div, lg_dt, open_q = get_give_school_info(school)
                
                temp = [abbr, div, year, place, True, score, school, ID, lat, long, exact, size, conf, type_var, county, brd_div, lg_dt, open_q]
                csv_writer.writerow(temp)

bkb
fb
Wall time: 16 s


We also read in the calculated average population center of Illinois as per the Census data and display the interpolated information below. As this process was very short and straightforward, the data was recorded by hand and calculated in Excel; each decade of Census information broken down by county was converted into a percent of the state's population and multiplied by the latitude and longitude of the county, and the sum of the results revealed the decade-by-decade information contained in il_pop_data.xlsx.

In [8]:
il_pop = pd.read_excel('il_pop_data.xlsx', sheet_name='results')
il_pop.rename(columns = {'Unnamed: 0':'Year', 'lat':'Lat', 'long':'Long'}, inplace = True)
pre_il_pop = il_pop.copy()
temp = []
for year in range(1900, 2011):
    if year % 10 == 0: temp.append(il_pop.iloc[int((year-1900)/10)].to_numpy())
    else: temp.append(np.array([np.nan, np.nan, np.nan]))
il_pop = pd.DataFrame(temp, columns = ["Year", "Lat", "Long"]).interpolate()
il_pop["Year"] = il_pop["Year"].astype(int)
display(il_pop)

,Year,Lat,Long
0,1900,40.719781,-88.712663
1,1901,40.727932,-88.705593
2,1902,40.736083,-88.698522
3,1903,40.744234,-88.691452
4,1904,40.752385,-88.684382
...,...,...,...
106,2006,41.187110,-88.453889
107,2007,41.186764,-88.454712
108,2008,41.186417,-88.455536
109,2009,41.186070,-88.456360


### Cleaning
We first observe what the dataset looks like and check which rows do not have latitude information, check that they are the exact same rows that do not have longitude information, and drop those entries. The new data set is called ihsa_tidy and will be the data set that we use for all analyses.

In [9]:
ihsa_raw = pd.read_csv('IHSA_Deposit.csv', keep_default_na = False, na_values = '', dtype = {"YEAR":int, "LAT":float, "LONG":float})
display(ihsa_raw.head())
print(ihsa_raw[ihsa_raw["LAT"].isna()].equals(ihsa_raw[ihsa_raw["LAT"].isna()]))
ihsa_raw_problems = ihsa_raw[ihsa_raw["LAT"].isna()]
display(ihsa_raw_problems[["ABBR", "DIV", "YEAR", "PLACE", "SCORE", "SCHOOL"]])

,ABBR,DIV,YEAR,PLACE,TOURNAMENT,SCORE,SCHOOL,ID,LAT,LONG,EXACT,SIZE,CONF,TYPE,COUNTY,BRD-DIV,LG-DT,OPEN
0,bwb,N/A,2003,1,False,13064.0,Rockford (Jefferson),1824.0,42.2074,-89.0377,True,1656.0,Northern Illinois (NIC-10),Public Coed,Winnebago,4.0,10.0,True
1,bwb,N/A,2003,2,False,12987.0,Freeport (H.S.),619.0,42.2886,-89.6288,True,1173.5,Northern Illinois (NIC-10),Public Coed,Stephenson,4.0,10.0,True
2,bwb,N/A,2003,3,False,12667.0,Collinsville,343.0,38.6469,-90.0110,True,1904.5,Southwestern,Public Coed,St. Clair,7.0,19.0,True
3,bwb,N/A,2003,4,False,12665.0,Oak Lawn (Community),1502.0,41.7212,-87.7678,True,1774.5,South Suburban,Public Coed,Cook,3.0,8.0,True
4,bwb,N/A,2003,5,False,12664.0,Alton (Sr.),110.0,38.9364,-90.1581,True,2008.0,Southwestern,Public Coed,Madison,7.0,19.0,True


True


,ABBR,DIV,YEAR,PLACE,SCORE,SCHOOL
1971,gob,A,1983,2,NaN,vacated
9437,chc,C,2015,5,NaN,vacated
16482,mu,A,1991,9,576.0,unknown (NOT Ottawa)
17539,mu,B,2012,3,792.0,vacated


In [10]:
ihsa_tidy = ihsa_raw.drop(ihsa_raw[(ihsa_raw["ABBR"] == "WR") & (ihsa_raw["YEAR"].astype(int) >= 1984)].index)
print(ihsa_tidy.shape)

(36074, 18)


We create a helper method that adds a column that shows the relative rank of the division (1 is the smallest, up to however many divisions there were for that activity in that year) and the rank of the largest division, expressed a fraction passed as a string. Note that 1/2 is not equal to 2/4; 1/2 means that division was the smallest of two, 2/4 means that it was the 2nd smallest of 4.

In [11]:
def divpts(div, abbr, year):
    if div == "N/A": divpts = 0
    elif div == "A":
        if abbr not in ["TRB", "MU"]: divpts = 1
        elif abbr == "TRB":
            if year > 1926: divpts = 1
            elif year != 1916: divpts = 2
            else: divpts = 3
        else: divpts = 4
    elif div == "AA":
        if abbr !="MU": divpts=2
        else: divpts = 5
    elif re.match("\d", div[0]): divpts = int(div[0])
    elif div == "B":
        if abbr == "TRB":
            if year != 1916: divpts = 1
            else: divpts = 2
        else:
            divpts = 3
    elif div == "C" and abbr == "CHC": divpts = 0
    elif div == "C" and abbr == "TRB": divpts = 1
    elif div == "C": divpts = 2
    elif div == "D": divpts = 1
    elif div == "S": divpts = 1
    elif div == "M": divpts = 2
    elif div == "L": divpts = 3
    else: divpts = -1
    return divpts
    
def global_div(div, abbr, year):
    this_pts = divpts(div, abbr, year)
    maxpts = ihsa_tidy[(ihsa_tidy["ABBR"]==abbr) & (ihsa_tidy["YEAR"]==year)]["DIV"].copy().apply(lambda x: divpts(x, abbr, year)).max()
    if maxpts == -1: return "-1"
    if this_pts == 0: return "0"
    return str(this_pts) + "/" + str(maxpts)

ihsa_tidy["ABBR"] = ihsa_tidy["ABBR"].astype(str).apply(lambda x: x.upper())
ihsa_tidy["YEAR"] = ihsa_tidy["YEAR"].astype(int)
ihsa_tidy = ihsa_tidy[~ ihsa_tidy["LAT"].isna()]
ihsa_tidy["GL_DIV"] = ihsa_tidy.apply(lambda x: global_div(x["DIV"], x["ABBR"], x["YEAR"]), axis = 1)

ihsa_tidy.to_csv("Working_IHSA_Deposit.csv")

This last ihsa_tidy is the dataset used by the rest of the project.